In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import pandas as pd

# CSV 파일 읽어오기
circulatory_df = pd.read_csv('content/circulatory_diseases_combined.csv')
patients_df = pd.read_csv('content/patients.csv')

# subject_id를 기준으로 두 데이터프레임을 병합
merged_df = pd.merge(circulatory_df, patients_df, on='subject_id', how='inner')

# 병합된 데이터를 patients_icu.csv 파일로 저장
merged_df.to_csv('content/patients_icu.csv', index=False)

In [ ]:
chartevents = pd.read_csv("/content/drive/MyDrive/DA_dataset/6hour_chartevents.csv")
#inputevents = pd.read_csv("/content/drive/MyDrive/DA_dataset/6hour_inputevents.csv")
#outputevents = pd.read_csv("/content/drive/MyDrive/DA_dataset/6hour_outputevents.csv")
patients = pd.read_csv("/content/drive/MyDrive/DA_dataset/patients_icu.csv")
d_items = pd.read_csv("/content/drive/MyDrive/DA_dataset/d_items.csv")
icustays = pd.read_csv("/content/drive/MyDrive/DA_dataset/icustays.csv")
diagnoses = pd.read_csv("/content/drive/MyDrive/DA_dataset/diagnoses_icd.csv")
d_icd = pd.read_csv("/content/drive/MyDrive/DA_dataset/d_icd_diagnoses.csv")

print(chartevents.shape)
#print(inputevents.shape)
#print(outputevents.shape)
print(patients.shape)
print(d_items.shape)
print(icustays.shape)
print(diagnoses.shape)
print(d_icd.shape)

<ipython-input-5-8fe1a5a5afde>:1: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  chartevents = pd.read_csv("/content/drive/MyDrive/DA_dataset/6hour_chartevents.csv")


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
chartevents_m = pd.merge(chartevents,d_items, on = 'itemid')
#inputevents_m = pd.merge(inputevents,d_items, on = 'itemid')
#outputevents_m = pd.merge(outputevents,d_items, on = 'itemid')
#이벤트의 이름 확인을 위해 d_items와 병합

In [ ]:
# 화씨를 측정한 이벤트는 섭씨로 변환함 -> 섭씨를 측정한 이벤트 사용
chartevents_m.loc[chartevents_m['label'] == 'Temperature Fahrenheit','valuenum'] = chartevents_m.loc[chartevents_m['label'] == 'Temperature Fahrenheit','valuenum'].map(lambda x : (x - 32) * 5.0 / 9.0)
chartevents_m.loc[chartevents_m['label'] == 'Temperature Fahrenheit','valuenum']
# 아래는 변환 결과

44          36.555556
802         36.611111
883         36.611111
930         36.944444
1028        36.944444
              ...    
27539231    36.611111
27539328    36.666667
27539383    37.277778
27539506    36.055556
27539559    35.555556
Name: valuenum, Length: 142559, dtype: float64

In [ ]:
chart = chartevents_m[['subject_id','stay_id','charttime','itemid','value','valuenum',"valueuom",'intime','label']]
chart.to_csv('content/circulator_chart_6.csv', index=False)
print(chart)

          subject_id   stay_id            charttime  itemid      value  \
0           10003700  30600691  2165-04-24 06:00:00  220045       56.0   
1           10003700  30600691  2165-04-24 06:00:00  220179      126.0   
2           10003700  30600691  2165-04-24 06:00:00  220180       78.0   
3           10003700  30600691  2165-04-24 06:00:00  220181       88.0   
4           10003700  30600691  2165-04-24 06:00:00  220210       14.0   
...              ...       ...                  ...     ...        ...   
29979877    19999068  30143796  2161-08-24 11:00:00  228404  Withdraws   
29979878    19999068  30143796  2161-08-24 11:00:00  228405  Withdraws   
29979879    19999068  30143796  2161-08-24 11:00:00  228406  Withdraws   
29979880    19999068  30143796  2161-08-24 11:00:00  228407  Withdraws   
29979881    19999068  30143796  2161-08-24 11:00:00  228414     Normal   

          valuenum  valueuom               intime  \
0             56.0       bpm  2165-04-24 05:43:00   
1    

In [ ]:
import pandas as pd

# 1. patients_icu에서 hadm_id를 추출
icu_hadm_ids = patients['hadm_id']

# 2. icustays 데이터에서 hadm_id가 patients_icu에 있는 hadm_id만 필터링
filtered_icustays = icustays[icustays['hadm_id'].isin(icu_hadm_ids)]

# 3. los 컬럼에 대한 요약 통계 출력
filtered_icustays['los'].describe()

count    25431.000000
mean         3.039147
std          3.116626
min          0.250243
25%          1.166753
50%          1.995093
75%          3.570579
max         21.056007
Name: los, dtype: float64

In [ ]:
icustays = icustays[~((icustays['los'] >  21.07) | (icustays['los'] < 0.25))]
stay_id = icustays['stay_id']
#6시간보다 빨리 퇴원하거나, 입실기간이 2표준편차 이상인 이상치 제거
chart = chart[chart['stay_id'].isin(stay_id)]
chart
chart.to_csv('content/circulator_chart_6.csv', index=False)

In [ ]:
import pandas as pd
chart = pd.read_csv('content/circulator_chart_6.csv')

C:\Users\Eugene Clark\AppData\Local\Temp\ipykernel_24036\3106247094.py:2: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  chart = pd.read_csv('content/circulator_chart_6.csv')


In [ ]:
# 'label' 컬럼에서 결측치를 제외하고 유니크 값들의 빈도수를 계산하고 상위 20개를 나열
top_20_labels = chart['label'].value_counts(dropna=True).head(50)

# 결과 출력
print(top_20_labels)

label
Safety Measures                                  779300
Heart Rate                                       615249
Respiratory Rate                                 607510
O2 saturation pulseoxymetry                      595621
Heart Rhythm                                     474283
Non Invasive Blood Pressure systolic             407782
Non Invasive Blood Pressure diastolic            407663
Non Invasive Blood Pressure mean                 407361
Ectopy Type 1                                    399224
Orientation                                      237394
Education Topic                                  192794
Less Restrictive Measures                        190838
Turn                                             180000
Head of Bed                                      178455
Arterial Blood Pressure mean                     174643
Arterial Blood Pressure systolic                 173109
Arterial Blood Pressure diastolic                173091
Temperature Site                          

In [ ]:
# 해당 라벨값의 행 조회
chart[(chart['label']=='Heart Rhythm')].head

<bound method NDFrame.head of           subject_id   stay_id            charttime  itemid  \
1345450     10004235  34100191  2196-02-24 17:16:00  220048   
1345514     10004235  34100191  2196-02-24 18:00:00  220048   
1345598     10004235  34100191  2196-02-24 19:00:00  220048   
1345614     10004235  34100191  2196-02-24 20:00:00  220048   
1345757     10004235  34100191  2196-02-24 21:00:00  220048   
...              ...       ...                  ...     ...   
29303284    19999068  30143796  2161-08-24 07:00:00  220048   
29303302    19999068  30143796  2161-08-24 08:00:00  220048   
29303436    19999068  30143796  2161-08-24 09:00:00  220048   
29303453    19999068  30143796  2161-08-24 10:00:00  220048   
29303487    19999068  30143796  2161-08-24 11:00:00  220048   

                             value  valuenum valueuom               intime  \
1345450   AF (Atrial Fibrillation)       NaN      NaN  2196-02-24 17:07:00   
1345514   AF (Atrial Fibrillation)       NaN      NaN  21

In [ ]:
import pandas as pd

# 이벤트 그룹 정의 -> 각 측정할 값에 해당하는 이벤트들
hr_events = ["Heart Rate", "Heart rate Alarm - High", "Heart Rate Alarm - Low"]
systolic_bp_events = ["Arterial Blood Pressure systolic", "ART BP Systolic", "Non Invasive Blood Pressure systolic"]
diastolic_bp_events = ['Arterial Blood Pressure diastolic', 'ART BP Diastolic', 'Non Invasive Blood Pressure diastolic']
Blood_Glucose = ["Glucose (serum)", "Glucose (whole blood)", 'Glucose finger stick (range 70-100)']
O2_Saturation_Pulseoxymetry = ["O2 saturation pulseoxymetry"]
temperature = ["Temperature Fahrenheit", "Temperature Celsius"]
Respiratory_Rate = ['Respiratory Rate']
PH = ['PH (Arterial)', 'PH (Venous)']
GCS_Eye = ['GCS - Eye Opening']
GCS_Verbal = ['GCS - Verbal Response']
GCS_Motor = ['GCS - Motor Response']
Arterial_Blood_Pressure_mean = ['Arterial Blood Pressure mean']
#i1 = ['Arterial Blood Pressure mean']
#i2 = 'Arterial Blood Pressure mean'

# 범주형 변수 이벤트 그룹
categorical_events = ['Pain Assessment Method', 'Activity Tolerance', 'Head of Bed']

# 필터링할 이벤트만 있는 행 선택
filtered_df = chart[chart['label'].isin(hr_events + systolic_bp_events + diastolic_bp_events +
                                        Blood_Glucose + O2_Saturation_Pulseoxymetry + temperature + Respiratory_Rate +
                                        PH + GCS_Eye + GCS_Verbal + GCS_Motor + categorical_events)].copy()

# intime 컬럼을 datetime 형식으로 변환 -> 시간 계산을 위해 변경
filtered_df['intime'] = pd.to_datetime(filtered_df['intime'])
filtered_df['charttime'] = pd.to_datetime(filtered_df['charttime'])

# 입실 후 시간 단위로 구분 -> 각 이벤트들을 1시간동안 이벤트인지 2시간 동안 이벤트인지 설정하여 새로운 컬럼으로 만듬
filtered_df['time_since_admission'] = (filtered_df['charttime'] - filtered_df['intime']).dt.total_seconds() / 3600
filtered_df['time_bin'] = filtered_df['time_since_admission'].apply(lambda x: int(x) if x < 6 else 5)  # 0~5시간까지만 포함

# 이벤트 그룹별 레이블 지정 함수
def get_event_group(label):
    if label in hr_events:
        return 'Heart Rate'
    elif label in systolic_bp_events:
        return 'Systolic BP'
    elif label in diastolic_bp_events:
        return 'Diastolic BP'
    elif label in Blood_Glucose:
        return 'Blood Glucose'
    elif label in O2_Saturation_Pulseoxymetry:
        return 'O2 Saturation Pulseoxymetry'
    elif label in temperature:
        return 'Temperature'
    elif label in Respiratory_Rate:
        return 'Respiratory Rate'
    elif label in PH:
        return 'Arterial PH'
    elif label in GCS_Eye:
        return 'GCS_Eye'
    elif label in GCS_Verbal:
        return 'GCS_Verbal'
    elif label in GCS_Motor:
        return 'GCS_Motor'
    elif label in categorical_events:
        return label
    else:
        return 'Other'

# 그룹으로 분류하여 새로운 컬럼으로 만듦
filtered_df['event_group'] = filtered_df['label'].apply(get_event_group)

# 수치형 변수 처리
numerical_events = hr_events + systolic_bp_events + diastolic_bp_events + Blood_Glucose + O2_Saturation_Pulseoxymetry + temperature + Respiratory_Rate + PH + GCS_Eye + GCS_Verbal + GCS_Motor

numerical_df = filtered_df[filtered_df['label'].isin(numerical_events)].copy()

# 범주형 변수 처리
categorical_df = filtered_df[filtered_df['label'].isin(categorical_events)].copy()

# 수치형 변수에 대해 평균 계산
numerical_pivot = (numerical_df.groupby(['stay_id', 'time_bin', 'event_group'])
                   .agg({'valuenum': 'mean'})
                   .reset_index()
                   .pivot(index=['stay_id', 'time_bin'], columns='event_group', values='valuenum')
                   .reset_index())


# 범주형 변수에서 최빈값 계산 (각 레이블에 맞게 컬럼명 지정)
def get_mode(df, col, value_col):
    """범주형 변수에서 최빈값을 계산"""
    mode_df = df[df['label'] == col].groupby(['stay_id', 'time_bin'])[value_col] \
        .agg(lambda x: x.mode()[0] if not x.mode().empty else None).reset_index()

    # 최빈값이 계산된 컬럼을 레이블로 변경
    mode_df = mode_df.rename(columns={value_col: col})

    return mode_df

# 각 범주형 변수에 대해 최빈값을 계산하고 병합
mode_dfs = []
for categorical in categorical_events:
    mode_df = get_mode(filtered_df, categorical, 'value')
    mode_dfs.append(mode_df)

# 모든 범주형 데이터 최빈값을 하나의 데이터프레임으로 결합
mode_combined_df = mode_dfs[0]
for df in mode_dfs[1:]:
    mode_combined_df = pd.merge(mode_combined_df, df, on=['stay_id', 'time_bin'], how='left')

# 수치형 데이터와 범주형 데이터 병합
final_df = pd.merge(numerical_pivot, mode_combined_df, on=['stay_id', 'time_bin'], how='left')

# stay_id별로 time_bin의 고유 값 개수를 계산
time_bin_counts = final_df.groupby('stay_id')['time_bin'].nunique().reset_index()
time_bin_counts = time_bin_counts.rename(columns={'time_bin': 'time_bin_count'})

# time_bin이 6개 (0~5)인 stay_id만 선택
valid_stay_ids = time_bin_counts[time_bin_counts['time_bin_count'] == 6]['stay_id']

# 조건에 맞는 stay_id만 유지
final_df = final_df[final_df['stay_id'].isin(valid_stay_ids)].copy()

# 결과 확인
final_df


,stay_id,time_bin,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed
0,30000153,0,NaN,NaN,77.0,3.0,5.0,1.0,104.000000,100.000000,17.000000,113.0,NaN,Change in Vitals,Tolerated Well,Reverse T-berg
1,30000153,1,7.30,167.0,66.5,NaN,NaN,NaN,NaN,NaN,16.000000,141.0,37.277778,NaN,NaN,NaN
2,30000153,2,NaN,NaN,NaN,NaN,NaN,NaN,83.000000,100.000000,NaN,NaN,NaN,NaN,NaN,NaN
3,30000153,3,7.31,183.5,60.0,NaN,NaN,NaN,87.500000,100.000000,15.000000,116.0,37.500000,NaN,NaN,NaN
4,30000153,4,NaN,NaN,56.0,4.0,6.0,1.0,103.000000,100.000000,20.000000,111.0,NaN,Non-verbal Cues,Tolerated Well,Reverse T-berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420710,39999810,1,NaN,NaN,78.5,2.0,6.0,5.0,74.200000,96.333333,16.333333,129.5,NaN,NaN,NaN,NaN
420711,39999810,2,NaN,NaN,75.0,NaN,NaN,NaN,62.000000,96.000000,15.000000,126.0,NaN,NaN,NaN,NaN
420712,39999810,3,NaN,NaN,74.0,3.0,6.0,5.0,81.666667,96.000000,24.000000,137.0,36.666667,NaN,NaN,NaN
420713,39999810,4,NaN,NaN,70.0,NaN,NaN,NaN,62.000000,96.000000,15.000000,118.0,NaN,NaN,NaN,NaN


In [ ]:
final_df.to_csv('final_df_v1_dataset.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 유효하지 않은 stay_id가 없는지 확인
print(final_df['stay_id'].nunique() == valid_stay_ids.nunique())

True


In [ ]:
#결측치 비율 출력
final_df.isnull().sum() / len(final_df)

stay_id                        0.000000
time_bin                       0.000000
Arterial PH                    0.864546
Blood Glucose                  0.734726
Diastolic BP                   0.063568
GCS_Eye                        0.632158
GCS_Motor                      0.634458
GCS_Verbal                     0.633079
Heart Rate                     0.036723
O2 Saturation Pulseoxymetry    0.053377
Respiratory Rate               0.045890
Systolic BP                    0.063419
Temperature                    0.649067
Pain Assessment Method         0.691211
Activity Tolerance             0.801162
Head of Bed                    0.785808
dtype: float64

In [ ]:
# 각 환자들이 최소 1번 이상 해당 라벨의 검사지표가 있는지


def calculate_label_experience_ratio(final_df, label_column='Diastolic BP'):
    # 라벨이 1번 이상 기록된 환자만 필터링
    label_count_per_patient = final_df.groupby(['stay_id', label_column]).size().reset_index(name='count')

    # 라벨이 1번 이상 기록된 환자만 필터링
    patients_with_label = label_count_per_patient[label_count_per_patient['count'] > 0]

    # 환자별 라벨이 1번 이상 존재한 것 확인 (각 환자별 라벨이 기록된 개수)
    patients_with_label_summary = patients_with_label.groupby('stay_id')[label_column].nunique()

    # 각 환자(stay_id)별로 1번 이상 기록된 라벨 개수 출력
    #print(patients_with_label_summary)

    # 1번 이상 경험한 환자 수
    num_patients_with_label = len(patients_with_label_summary)

    # 전체 환자 수
    num_total_patients = len(final_df['stay_id'].unique())

    # 1번 이상 경험한 환자 비율 계산
    patients_with_label_ratio = num_patients_with_label / num_total_patients

    # 비율 출력
    #print(f"{label_column} 피검 환자 비율: {patients_with_label_ratio:.2f}")

    # 백분율로 출력
    patients_with_label_percentage = patients_with_label_ratio * 100
    print(f"{label_column} 피검 환자 비율: {patients_with_label_percentage:.2f}%")

    #return patients_with_label_ratio, patients_with_label_percentage


# 'stay_id' 컬럼 제외한 나머지 컬럼 리스트 얻기: 단 stay_id와 time_bin은 제외
columns_to_check = [col for col in final_df.columns if col != 'stay_id' and col != 'time_bin']

# 각 컬럼에 대해 함수 호출
for col in columns_to_check:
    calculate_label_experience_ratio(final_df, label_column=col)

Arterial PH 피검 환자 비율: 43.05%
Blood Glucose 피검 환자 비율: 82.65%
Diastolic BP 피검 환자 비율: 99.83%
GCS_Eye 피검 환자 비율: 97.91%
GCS_Motor 피검 환자 비율: 97.55%
GCS_Verbal 피검 환자 비율: 97.75%
Heart Rate 피검 환자 비율: 99.90%
O2 Saturation Pulseoxymetry 피검 환자 비율: 99.75%
Respiratory Rate 피검 환자 비율: 99.68%
Systolic BP 피검 환자 비율: 99.83%
Temperature 피검 환자 비율: 96.26%
Pain Assessment Method 피검 환자 비율: 91.89%
Activity Tolerance 피검 환자 비율: 76.27%
Head of Bed 피검 환자 비율: 80.87%


In [ ]:
#데이터 프레임 저장
#final_df.to_csv('average_calculation_per_hour_in_6hour.csv', index=False)

In [ ]:
len(final_df['stay_id'].unique()) # 총 환자

58106

In [ ]:
# 6시간동안 한번도 측정하지 않은 컬럼이 있는 환자는 제거 ->하나라도 있으면 보간가능
filtered_df = final_df.groupby('stay_id').filter(
    lambda group: not (group.isna().all(axis=0).any())
)
# 배열, 포인터, 구조체
# 결과 확인
filtered_df

,stay_id,time_bin,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed
0,30000153,0,NaN,NaN,77.0,3.0,5.0,1.0,104.000000,100.0,17.0,113.0,NaN,Change in Vitals,Tolerated Well,Reverse T-berg
1,30000153,1,7.30,167.0,66.5,NaN,NaN,NaN,NaN,NaN,16.0,141.0,37.277778,NaN,NaN,NaN
2,30000153,2,NaN,NaN,NaN,NaN,NaN,NaN,83.000000,100.0,NaN,NaN,NaN,NaN,NaN,NaN
3,30000153,3,7.31,183.5,60.0,NaN,NaN,NaN,87.500000,100.0,15.0,116.0,37.500000,NaN,NaN,NaN
4,30000153,4,NaN,NaN,56.0,4.0,6.0,1.0,103.000000,100.0,20.0,111.0,NaN,Non-verbal Cues,Tolerated Well,Reverse T-berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420688,39999301,1,NaN,114.0,68.0,NaN,NaN,NaN,60.000000,94.0,12.0,120.0,NaN,NaN,NaN,NaN
420689,39999301,2,NaN,NaN,66.0,NaN,NaN,NaN,56.000000,95.0,16.0,124.0,NaN,NaN,NaN,NaN
420690,39999301,3,NaN,NaN,63.0,NaN,NaN,NaN,58.000000,91.0,15.0,119.0,NaN,NaN,NaN,NaN
420691,39999301,4,7.42,NaN,57.0,4.0,6.0,5.0,75.666667,90.0,15.0,111.0,36.055556,Patient Verbalized,Tolerated Well,45 Degrees


In [ ]:
#결측치 비율 출력
filtered_df.isnull().sum() / len(filtered_df)

stay_id                        0.000000
time_bin                       0.000000
Arterial PH                    0.727059
Blood Glucose                  0.644632
Diastolic BP                   0.054551
GCS_Eye                        0.650124
GCS_Motor                      0.652112
GCS_Verbal                     0.650878
Heart Rate                     0.038511
O2 Saturation Pulseoxymetry    0.052717
Respiratory Rate               0.044407
Systolic BP                    0.054529
Temperature                    0.605280
Pain Assessment Method         0.654317
Activity Tolerance             0.747139
Head of Bed                    0.736733
dtype: float64

In [ ]:
len(filtered_df['stay_id'].unique()) # 총 환자수

15264

In [ ]:
#컬럼 리스트 얻기: 단 stay_id와 time_bin은 제외
columns_to_check = [col for col in filtered_df.columns if col != 'stay_id' and col != 'time_bin']

# 각 컬럼에 대해 함수 호출
for col in columns_to_check:
    calculate_label_experience_ratio(filtered_df, label_column=col)

Arterial PH 피검 환자 비율: 100.00%
Blood Glucose 피검 환자 비율: 100.00%
Diastolic BP 피검 환자 비율: 100.00%
GCS_Eye 피검 환자 비율: 100.00%
GCS_Motor 피검 환자 비율: 100.00%
GCS_Verbal 피검 환자 비율: 100.00%
Heart Rate 피검 환자 비율: 100.00%
O2 Saturation Pulseoxymetry 피검 환자 비율: 100.00%
Respiratory Rate 피검 환자 비율: 100.00%
Systolic BP 피검 환자 비율: 100.00%
Temperature 피검 환자 비율: 100.00%
Pain Assessment Method 피검 환자 비율: 100.00%
Activity Tolerance 피검 환자 비율: 100.00%
Head of Bed 피검 환자 비율: 100.00%


In [ ]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91584 entries, 0 to 420692
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   stay_id                      91584 non-null  int64  
 1   time_bin                     91584 non-null  int64  
 2   Arterial PH                  24997 non-null  float64
 3   Blood Glucose                32546 non-null  float64
 4   Diastolic BP                 86588 non-null  float64
 5   GCS_Eye                      32043 non-null  float64
 6   GCS_Motor                    31861 non-null  float64
 7   GCS_Verbal                   31974 non-null  float64
 8   Heart Rate                   88057 non-null  float64
 9   O2 Saturation Pulseoxymetry  86756 non-null  float64
 10  Respiratory Rate             87517 non-null  float64
 11  Systolic BP                  86590 non-null  float64
 12  Temperature                  36150 non-null  float64
 13  Pain Assessment Meth

In [ ]:
# 결측치 전진, 후진 보간

grouped = filtered_df.groupby('stay_id')

def fill_missing_values(group):
    # 기존 인덱스를 리셋하여 그룹화 키를 제거합니다
    group = group.reset_index(drop=True)

    # 'time_bin'을 인덱스로 설정합니다
    group = group.set_index('time_bin')

    # 1. 모든 변수의 결측치를 전진 채움으로 채웁니다
    group = group.ffill()

    # 2. 여전히 결측치가 남아있는 경우 후진 채움으로 채웁니다
    group = group.bfill()

    # 인덱스를 리셋합니다
    group = group.reset_index()

    return group

# 그룹화된 데이터에 결측치 처리 함수를 적용합니다
filtered_df = grouped.apply(fill_missing_values).reset_index(drop=True)

# 결과를 확인합니다
filtered_df

C:\Users\Eugene Clark\AppData\Local\Temp\ipykernel_24036\1690401265.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = grouped.apply(fill_missing_values).reset_index(drop=True)


,time_bin,stay_id,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed
0,0,30000153,7.30,167.0,77.0,3.0,5.0,1.0,104.000000,100.0,17.0,113.0,37.277778,Change in Vitals,Tolerated Well,Reverse T-berg
1,1,30000153,7.30,167.0,66.5,3.0,5.0,1.0,104.000000,100.0,16.0,141.0,37.277778,Change in Vitals,Tolerated Well,Reverse T-berg
2,2,30000153,7.30,167.0,66.5,3.0,5.0,1.0,83.000000,100.0,16.0,141.0,37.277778,Change in Vitals,Tolerated Well,Reverse T-berg
3,3,30000153,7.31,183.5,60.0,3.0,5.0,1.0,87.500000,100.0,15.0,116.0,37.500000,Change in Vitals,Tolerated Well,Reverse T-berg
4,4,30000153,7.31,183.5,56.0,4.0,6.0,1.0,103.000000,100.0,20.0,111.0,37.500000,Non-verbal Cues,Tolerated Well,Reverse T-berg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91579,1,39999301,7.42,114.0,68.0,4.0,6.0,5.0,60.000000,94.0,12.0,120.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees
91580,2,39999301,7.42,114.0,66.0,4.0,6.0,5.0,56.000000,95.0,16.0,124.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees
91581,3,39999301,7.42,114.0,63.0,4.0,6.0,5.0,58.000000,91.0,15.0,119.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees
91582,4,39999301,7.42,114.0,57.0,4.0,6.0,5.0,75.666667,90.0,15.0,111.0,36.055556,Patient Verbalized,Tolerated Well,45 Degrees


In [ ]:
filtered_df.to_csv('circulatory_diseases_dataset_6.csv', index=False, encoding='utf-8-sig')

In [ ]:
len(filtered_df['stay_id'].unique()) # 총 환자수

15264

In [ ]:
#환자의 나이 및 성별 포함
patients = patients[['subject_id', 'gender', 'anchor_age']]
personal = pd.merge(icustays[['subject_id','stay_id']],patients,on = 'subject_id')
personal = personal[['stay_id','gender','anchor_age']]
personal = personal.drop_duplicates()
personal

,stay_id,gender,anchor_age
0,34569476,F,67
2,39289362,F,49
4,39387567,M,57
5,32563675,F,72
8,35370343,M,48
...,...,...,...
51327,39987151,F,79
51328,39988534,F,48
51329,39988873,F,42
51331,39992247,M,72


In [ ]:
dataset = pd.merge(filtered_df,personal)
dataset

,time_bin,stay_id,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed,gender,anchor_age
0,0,30000213,7.37,125.0,62.5,2.0,4.0,1.0,79.500000,96.5,20.5,165.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
1,1,30000213,7.37,108.0,53.0,2.0,4.0,1.0,66.000000,97.0,23.0,126.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
2,2,30000213,7.37,126.5,67.0,4.0,6.0,4.0,83.000000,100.0,18.0,118.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
3,3,30000213,7.37,126.5,57.0,4.0,6.0,4.0,75.000000,99.0,26.0,128.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
4,4,30000213,7.37,126.5,56.0,4.0,6.0,4.0,79.000000,98.0,20.0,130.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,M,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32041,1,39999301,7.42,114.0,68.0,4.0,6.0,5.0,60.000000,94.0,12.0,120.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees,M,77
32042,2,39999301,7.42,114.0,66.0,4.0,6.0,5.0,56.000000,95.0,16.0,124.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees,M,77
32043,3,39999301,7.42,114.0,63.0,4.0,6.0,5.0,58.000000,91.0,15.0,119.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees,M,77
32044,4,39999301,7.42,114.0,57.0,4.0,6.0,5.0,75.666667,90.0,15.0,111.0,36.055556,Patient Verbalized,Tolerated Well,45 Degrees,M,77


In [ ]:
# 성별 인코딩
d = {'M' : 1 , 'F': 0 }
dataset['gender'] = dataset['gender'].map(d)
dataset

,time_bin,stay_id,Arterial PH,Blood Glucose,Diastolic BP,GCS_Eye,GCS_Motor,GCS_Verbal,Heart Rate,O2 Saturation Pulseoxymetry,Respiratory Rate,Systolic BP,Temperature,Pain Assessment Method,Activity Tolerance,Head of Bed,gender,anchor_age
0,0,30000213,7.37,125.0,62.5,2.0,4.0,1.0,79.500000,96.5,20.5,165.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
1,1,30000213,7.37,108.0,53.0,2.0,4.0,1.0,66.000000,97.0,23.0,126.0,36.333333,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
2,2,30000213,7.37,126.5,67.0,4.0,6.0,4.0,83.000000,100.0,18.0,118.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
3,3,30000213,7.37,126.5,57.0,4.0,6.0,4.0,75.000000,99.0,26.0,128.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
4,4,30000213,7.37,126.5,56.0,4.0,6.0,4.0,79.000000,98.0,20.0,130.0,36.444444,Non-verbal Cues,Tolerated Well,30 Degrees,1,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32041,1,39999301,7.42,114.0,68.0,4.0,6.0,5.0,60.000000,94.0,12.0,120.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees,1,77
32042,2,39999301,7.42,114.0,66.0,4.0,6.0,5.0,56.000000,95.0,16.0,124.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees,1,77
32043,3,39999301,7.42,114.0,63.0,4.0,6.0,5.0,58.000000,91.0,15.0,119.0,36.611111,Patient Verbalized,Tolerated Well,30 Degrees,1,77
32044,4,39999301,7.42,114.0,57.0,4.0,6.0,5.0,75.666667,90.0,15.0,111.0,36.055556,Patient Verbalized,Tolerated Well,45 Degrees,1,77


In [ ]:
filtered_df = dataset
filtered_df.to_csv('circulatory_diseases_dataset_6.csv', index=False, encoding='utf-8-sig')

In [ ]:
import pandas as pd

# 1. circulatory_diseases_dataset.csv 파일 읽기
circulatory_df = pd.read_csv('circulatory_diseases_dataset_6.csv')

# 2. patients_icu.csv 파일 읽기
icu_df = pd.read_csv('content/patients_icu.csv')

# 3. 두 데이터셋을 stay_id 컬럼을 기준으로 조인
merged_df = pd.merge(circulatory_df[['stay_id']], icu_df[['stay_id', 'los']], on='stay_id', how="inner")

# 4. stay_id와 los 컬럼만 선택
result_df = merged_df[['stay_id', 'los']]

# 결과 출력
print(result_df)

        stay_id       los
0      30000213  1.635278
1      30000213  1.635278
2      30000213  1.635278
3      30000213  1.635278
4      30000213  1.635278
...         ...       ...
23827  39999301  1.146910
23828  39999301  1.146910
23829  39999301  1.146910
23830  39999301  1.146910
23831  39999301  1.146910

[23832 rows x 2 columns]


In [ ]:
len(result_df['stay_id'].unique()) # 총 환자수

3972

In [ ]:
#결측치 비율 출력
result_df.isnull().sum() / len(result_df)

stay_id    0.0
los        0.0
dtype: float64

In [ ]:
# 타켓 데이터셋 추출
result_df.to_csv('circulatory_target_in_6hour.csv', index=False, encoding='utf-8-sig')
print(result_df)

        stay_id       los
0      30000213  1.635278
1      30000213  1.635278
2      30000213  1.635278
3      30000213  1.635278
4      30000213  1.635278
...         ...       ...
23827  39999301  1.146910
23828  39999301  1.146910
23829  39999301  1.146910
23830  39999301  1.146910
23831  39999301  1.146910

[23832 rows x 2 columns]


In [ ]:
# los값이 결측치인 환자 제거 후 csv 추출
filtered_circulatory_df = circulatory_df[circulatory_df['stay_id'].isin(result_df['stay_id'])]

print(filtered_circulatory_df)
filtered_circulatory_df.to_csv('circulatory_dataset_in_6hour.csv', index=False, encoding='utf-8-sig')

       time_bin   stay_id  Arterial PH  Blood Glucose  Diastolic BP  GCS_Eye  \
0             0  30000213         7.37          125.0          62.5      2.0   
1             1  30000213         7.37          108.0          53.0      2.0   
2             2  30000213         7.37          126.5          67.0      4.0   
3             3  30000213         7.37          126.5          57.0      4.0   
4             4  30000213         7.37          126.5          56.0      4.0   
...         ...       ...          ...            ...           ...      ...   
32041         1  39999301         7.42          114.0          68.0      4.0   
32042         2  39999301         7.42          114.0          66.0      4.0   
32043         3  39999301         7.42          114.0          63.0      4.0   
32044         4  39999301         7.42          114.0          57.0      4.0   
32045         5  39999301         7.42          114.0          66.0      4.0   

       GCS_Motor  GCS_Verbal  Heart Rat

In [ ]:
len(filtered_circulatory_df['stay_id'].unique()) # 총 환자수

3972

In [ ]:
chart_df = filtered_circulatory_df

# 고유값 조회
#o2_device_unique = chart_df['O2 Delivery Device(s)'].unique()
pain_assessment_method_unique = chart_df['Pain Assessment Method'].unique()
#pain_location_unique = chart_df['Pain Location'].unique()
activity_tolerance_unique = chart_df['Activity Tolerance'].unique()
#heart_rhythm_unique = chart_df['Heart Rhythm'].unique()
head_of_bed_unique = chart_df['Head of Bed'].unique()

# 결과 출력
#print("O2 Delivery Device(s) - Unique Values:", o2_device_unique)
print("Pain Assessment Method - Unique Values:", pain_assessment_method_unique)
#print("Pain Location - Unique Values:", pain_location_unique)
print("Activity Tolerance - Unique Values:", activity_tolerance_unique)
#print("Heart Rhythm - Unique Values:", heart_rhythm_unique)
print("Head of Bed - Unique Values:", head_of_bed_unique)

Pain Assessment Method - Unique Values: ['Non-verbal Cues' 'Patient Verbalized' 'Grimace Scale' 'Change in Vitals'
 '---' 'Wong-Baker Scale' 'Not applicable' 'FLACC' 'PAINAD']
Activity Tolerance - Unique Values: ['Tolerated Well' 'Good' 'Fair' 'Poor']
Head of Bed - Unique Values: ['30 Degrees' '45 Degrees' 'Flat in Bed' '15 Degrees' 'Reverse T-berg'
 '90 Degrees' '---' 'Trendelenberg']


In [ ]:
import pandas as pd

chart_df = filtered_circulatory_df.copy()  # 원본 데이터 유지
# 컬럼 이름을 리스트로 저장
columns_to_clean = ['Pain Assessment Method',
                    'Activity Tolerance', 'Head of Bed']

# 공백 제거 및 전진/후진 채움 적용
for column in columns_to_clean:
    # 공백 제거 (양쪽 공백)
    chart_df[column] = chart_df[column].str.strip()

    # '---' 값을 NaN으로 대체 (결측치 처리)
    chart_df[column] = chart_df[column].replace('---', pd.NA)

    # 전진 채움 적용
    chart_df[column] = chart_df[column].ffill()

    # 후진 채움 적용
    chart_df[column] = chart_df[column].bfill()

# 결과 확인
print(chart_df[columns_to_clean].head())

  Pain Assessment Method Activity Tolerance Head of Bed
0        Non-verbal Cues     Tolerated Well  30 Degrees
1        Non-verbal Cues     Tolerated Well  30 Degrees
2        Non-verbal Cues     Tolerated Well  30 Degrees
3        Non-verbal Cues     Tolerated Well  30 Degrees
4        Non-verbal Cues     Tolerated Well  30 Degrees


In [ ]:
# 고유값 조회
#o2_device_unique = chart_df['O2 Delivery Device(s)'].unique()
pain_assessment_method_unique = chart_df['Pain Assessment Method'].unique()
#pain_location_unique = chart_df['Pain Location'].unique()
activity_tolerance_unique = chart_df['Activity Tolerance'].unique()
#heart_rhythm_unique = chart_df['Heart Rhythm'].unique()
head_of_bed_unique = chart_df['Head of Bed'].unique()

# 결과 출력
#print("O2 Delivery Device(s) - Unique Values:", o2_device_unique)
print("Pain Assessment Method - Unique Values:", pain_assessment_method_unique)
#print("Pain Location - Unique Values:", pain_location_unique)
print("Activity Tolerance - Unique Values:", activity_tolerance_unique)
#print("Heart Rhythm - Unique Values:", heart_rhythm_unique)
print("Head of Bed - Unique Values:", head_of_bed_unique)

Pain Assessment Method - Unique Values: ['Non-verbal Cues' 'Patient Verbalized' 'Grimace Scale' 'Change in Vitals'
 'Wong-Baker Scale' 'Not applicable' 'FLACC' 'PAINAD']
Activity Tolerance - Unique Values: ['Tolerated Well' 'Good' 'Fair' 'Poor']
Head of Bed - Unique Values: ['30 Degrees' '45 Degrees' 'Flat in Bed' '15 Degrees' 'Reverse T-berg'
 '90 Degrees' 'Trendelenberg']


In [ ]:
# 최종 6시간 데이터셋
chart_df.to_csv('circulatory_dataset_in_6hour.csv', index=False, encoding='utf-8-sig')

In [ ]:
# 마지막 점검: 환자수
len(chart_df['stay_id'].unique()) # 총 환자수

3972

In [ ]:
df111 = pd.read_csv("circulatory_target_in_6hour.csv")

# stay_id와 los 컬럼이 같은 행을 제거
df111_unique = df111.drop_duplicates(subset=['stay_id', 'los'])

print(df111_unique)
df111_unique.to_csv('circulatory_target_in_6hour1.csv', index=False, encoding='utf-8-sig')

        stay_id       los
0      30000213  1.635278
6      30002415  1.014560
12     30004798  0.820937
18     30007175  2.967627
24     30007983  1.728009
...         ...       ...
23802  39972327  5.100868
23808  39980832  1.668796
23814  39983707  1.244919
23820  39988873  1.590185
23826  39999301  1.146910

[3972 rows x 2 columns]


In [ ]:
df12 = pd.read_csv('circulatory_dataset_in_6hour.csv')
time_bin_check = df12.groupby('stay_id')['time_bin'].nunique()

# time_bin이 6개(0~5)가 아닌 stay_id가 있는지 확인
invalid_stay_ids = time_bin_check[time_bin_check != 6]

print("유효하지 않은 stay_id 존재 여부:", not invalid_stay_ids.empty)

유효하지 않은 stay_id 존재 여부: False
